In [ ]:
# This notebook has the script to read the log files, preprocess them, 
# make visulaization plots and then create datasets to be used 
# by the ML algorithms

In [ ]:
# reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import matplotlib.gridspec as gridspec
import time
from decimal import *
import numpy as np
import csv
import time
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams.update({'font.size': 22, 'figure.autolayout' : True})

In [ ]:
def expand_cols_to_step_size(cols, steps):
    if steps == 1:
        return cols
    #else
    expanded_cols = []
    for col in cols:
        expanded_cols.extend([col+'_'+str(i) for i in range(steps)])
    return expanded_cols

In [ ]:
#=========================================
# Initial settings  
#=========================================

n_step_history = True
num_steps = 5
feat_step_size = '10ms'

wind_parse_dataset=True
create_dataset=True
plot_single_file_metrics=False
time_step_size = '50ms'
data_dir = '/proj/ngan/data-measurements/dataset_january2023/50ms_delay/'

# All values above this will be discarded to make the histograms look more readable
# It is NOT being used to filter out samples that go into the dataset  
ul_delay_outlier_threshold = 50 # ms

# files in ran_logs that we are interested in 
files = ['ulSchedGrantEvent.tsv', 'dlSchedGrantEvent.tsv', 'ABF_BeamMgmtWbm.tsv', 
         'ABF_BeamMgmtNbm.tsv', 
         'ABF_NrLegAdditionSuccessEvent.tsv',
         'ABF_NrLegAdditionMessagesStats.tsv',
         'dlLegSwitchEvent.tsv',
         'dlLegSwitchStatus.tsv'] # This is used to know when leg switching happens 
         #'ABF_BeamMgmtWbmNbm.tsv'] # Does not exist for position 2. Skipping for now 

# These features are being dropped so that time related features are not picked up to correlate with performance  
drop_features = ['absTime', 'absTime0', 'timeStamp', 'ver', 'bbUeRef', 'esfn', 'pucchSfn'] 
# These features are in some scenarios but not all and hence need to be removed
drop_features_after_processing = ['wbm_loadUe_ueTraceIdMsw', 'nbm_loadUe_ueTraceIdMsw', 'dl_loadUe_ueTraceIdMsw']
# cRnti and ueTraceIdMsw will be dropped as well after separating measurement and loader UEs 
# Several other features are dropped in the learning algorithm script before using as input to training 

# List of all ran_logs files in case we want to look at any other files 
#files = ['ABF_NrLegAdditionMessagesStats.tsv', 'ABF_BeamMgmtWbChanged.tsv', 
#         'dlSchedGrantEvent.tsv', 'ABF_BeamMgmtWbmNbmRes.tsv', 
#         'ABF_BeamMgmtWbm.tsv', 'ABF_NrConnectedUEStats.tsv', 
#         'NrNodeConfig.tsv', 'ABF_BeamMgmtNbm.tsv', 
#         'ulSchedGrantEvent.tsv', 'ABF_BeamMgmtWbmNbm.tsv']

drop_cols = ['ul_beamIndex', 'ul_loadUe_beamIndex', # non-gen cols
             # constant cols
             'ul_startPrb', 'ul_isTransformPrecoding', 'ul_BSValueLcg0', 'ul_BSValueLcg1',
             'ul_BSValueLcg2', 'ul_BSValueLcg3', 'ul_BSValueLcg4', 'ul_BSValueLcg6',
             'ul_BSValueLcg7', 'ul_BSRestimateLcg0', 'ul_BSRestimateLcg1',
             'ul_BSRestimateLcg2', 'ul_BSRestimateLcg3', 'ul_BSRestimateLcg4',
             'ul_BSRestimateLcg6', 'ul_BSRestimateLcg7', 'ul_VBit',
             'ul_wbUsedNbOverridden', 'ul_carrierAggregationUsed',
             'ul_clpcCarrierDemand', 'ul_isClpcSaturated', 'ul_totalCellsReqScheduling',
             'ul_linkAdaptationUeMode', 'ul_isDrxEnabled', 
             #--------
             'ul_loadUe_startPrb',
             'ul_loadUe_isTransformPrecoding', 'ul_loadUe_BSValueLcg0',
             'ul_loadUe_BSValueLcg1', 'ul_loadUe_BSValueLcg2', 'ul_loadUe_BSValueLcg3',
             'ul_loadUe_BSValueLcg4', 'ul_loadUe_BSValueLcg6', 'ul_loadUe_BSValueLcg7',
             'ul_loadUe_BSRestimateLcg0', 'ul_loadUe_BSRestimateLcg1',
             'ul_loadUe_BSRestimateLcg2', 'ul_loadUe_BSRestimateLcg3',
             'ul_loadUe_BSRestimateLcg4', 'ul_loadUe_BSRestimateLcg6',
             'ul_loadUe_BSRestimateLcg7', 'ul_loadUe_VBit',
             'ul_loadUe_wbUsedNbOverridden', 'ul_loadUe_carrierAggregationUsed',
             'ul_loadUe_clpcCarrierDemand', 'ul_loadUe_isClpcSaturated',
             'ul_loadUe_totalCellsReqScheduling', 'ul_loadUe_linkAdaptationUeMode',
             'ul_loadUe_isDrxEnabled']
drop_cols = expand_cols_to_step_size(drop_cols, num_steps)

In [ ]:

#===============================================================================
# Lists to aggregate raw metrics as an array over every file iterated through 
# This is done to create tables and plots after iterating through the log files
#===============================================================================

# BE CAREFUL some of these names are also being used for windowed metrics that are being plot after resampling
# CHECK and change these names if I want to revive using raw samples for the delay distribution 
# All windowed delays 
aggr_uldelay = []
aggr_dldelay = []

# All raw values of select input features  
aggr_ul_loadUe_measNumOfPrb = []
aggr_ul_loadUe_startPrb = []
aggr_ul_loadUe_numOfPrbs = []

# Windowed delay over Different UE devices
aggr_uldelay_sony = []
aggr_uldelay_wnc = []

# Windowed delays Different mobility cases 
aggr_uldelay_static = []
aggr_uldelay_moving = []

# Windowed delays Different pkt sizes 
# this is done per device so make sure that you are iterating over only one when you take the mean  
aggr_uldelay_1400 = []
aggr_uldelay_100 = []







# Resampled/windowed LTE ul delays for different UE devices 
# This needs to be done here because LTE samples will be removed from the dataset 
wind_aggr_uldelay_lte_wnc = np.array([])
wind_aggr_uldelay_lte_sony = np.array([])

# To aggregate raw MCS, CQI and RSRP metrics to make an overlay plot of these metrics over different loads
# For the proportion of bytes and PRBS per MCS graph 
aggr_mcs_stat_load10 = np.array([])
aggr_mcs_stat_load60 = np.array([])
aggr_mcs_stat_load60_1400B_m = np.array([])
aggr_mcs_stat_load60_100B_m = np.array([])
aggr_sinr0_stat_load10 = np.array([])
aggr_sinr1_stat_load10 = np.array([])
aggr_sinr0_stat_load60 = np.array([])
aggr_sinr1_stat_load60 = np.array([])
aggr_bytes_stat_load10 = np.array([])
aggr_bytes_stat_load60 = np.array([])
aggr_prbs_stat_load10 = np.array([])
aggr_prbs_stat_load60 = np.array([])

# Aggregate raw valaues of some input features
# For the histograms of SINR, BSR and network load over the entire dataset per device  
aggr_ul_m_WBeamRsrp_sony = np.array([])
aggr_ul_m_NBeamRsrp_sony = np.array([])
aggr_ul_m_sinrAchievable_sony = np.array([])
aggr_ul_m_BSRestimate_sony = np.array([])
aggr_ul_ld_macSduInBytes_sony = np.array([])
aggr_dl_m_sinr_sony = np.array([])

aggr_ul_m_WBeamRsrp_wnc = np.array([])
aggr_ul_m_NBeamRsrp_wnc = np.array([])
aggr_ul_m_sinrAchievable_wnc = np.array([])
aggr_ul_m_BSRestimate_wnc = np.array([])
aggr_ul_ld_macSduInBytes_wnc = np.array([])
aggr_dl_m_sinr_wnc = np.array([])

f = open('delay_percentiles_per_file.csv', 'w', newline='')
delay_percentiles_writer = csv.writer(f)
delay_percentiles_writer.writerow(['measUeDev', 'delayPktSize', 'position', 'load', '10_perc', '50_perc', '90_perc', '97_perc'])
#======================================================
start_time = time.time()
# separate datasets created for each ue_dev and delay_pkt_size 
for delayPktSize in delayPktSize_list: 
    for dev_id,measUeDev in enumerate(measUeDev_list):
        
        dataset = pd.DataFrame()
        # to gather per load stats for each chipset 
        # It is done over only one delay pkt size and one dev 
        per_load_ul_delay = [0,0,0,0,0,0,0]
        per_load_ul_txAtt = [0,0,0,0,0,0,0]
        per_load_ul_ack = [0,0,0,0,0,0,0]
        per_load_ul_buffer_meas = [0,0,0,0,0,0,0]
        per_load_ul_buffer_loader = [0,0,0,0,0,0,0]
        
        for pos_id,position in enumerate(position_list):
            # aggregate over load so that it can be plot on a single subplot
            for ld_id,load in enumerate(load_list):
                iteration_start_time = time.time()
                print('---------------------------------------------------------------------------------------------------------------------')
                print('delayPktSize: ', delayPktSize)
                print('measUeDev: ', measUeDev)
                print('position: ', position)
                print('load: ', load)
                print('---------------------------------------------------------------------------------------------------------------------')
                # read the delay and ran logs file
                delay_log_str = (data_dir+'delay_logs/load'+load+
                                 'Mbps_'+
                                 'delayPktSize'+delayPktSize+
                                 'B_pos'+position+
                                 '_'+measUeDev+'.csv')                    
                delay = pd.read_csv(delay_log_str, sep=',')
                print('Done Reading delay logs')
                # Read ran log CSV files. 
                ran_logs_dir = (data_dir+'ran_logs/load'+load+
                                'Mbps_delay_50_PktSize'+delayPktSize+
                                'B_pos'+position+'_'+
                                measUeDev+
                                '/logs/')
                # An extra tab at the end of each row is resulting in a NaN column. Remove that
                # Check if there are any rows with NaNs
                print('Reading from:'+
                      'load'+load+'Mbps_delayPktSize'+delayPktSize+
                      'B_pos'+position+'_'+measUeDev)
                ulSchedGrantEvent = pd.read_csv(ran_logs_dir+files[0], sep='\t')
                ulSchedGrantEvent.drop(ulSchedGrantEvent.columns[-1], axis=1, inplace=True)
                print('ulSchedGrantEvent Num rows: ', ulSchedGrantEvent.shape[0])
                #print(ulSchedGrantEvent.columns)
                #print("Number of rows with NaNs: " + str(np.count_nonzero(ulSchedGrantEvent.isnull())))  

                dlSchedGrantEvent = pd.read_csv(ran_logs_dir+files[1], sep='\t')
                dlSchedGrantEvent.drop(dlSchedGrantEvent.columns[-1], axis=1, inplace=True)
                print('dlSchedGrantEvent Num rows: ', dlSchedGrantEvent.shape[0])
                #print("Number of rows with NaNs: " + str(np.count_nonzero(dlSchedGrantEvent.isnull())))  
                
                ABF_BeamMgmtWbm = pd.read_csv(ran_logs_dir+files[2], sep='\t')
                ABF_BeamMgmtWbm.drop(ABF_BeamMgmtWbm.columns[-1], axis=1, inplace=True)
                ABF_BeamMgmtWbm = ABF_BeamMgmtWbm.rename(columns={'CRnti': 'cRnti'})
                print('ABF_BeamMgmtWbm Num rows: ', ABF_BeamMgmtWbm.shape[0])
                #print("Number of rows with NaNs: " + str(np.count_nonzero(ABF_BeamMgmtWbm.isnull())))  

                ABF_BeamMgmtNbm = pd.read_csv(ran_logs_dir+files[3], sep='\t')
                ABF_BeamMgmtNbm.drop(ABF_BeamMgmtNbm.columns[-1], axis=1, inplace=True)
                ABF_BeamMgmtNbm = ABF_BeamMgmtNbm.rename(columns={'CRnti': 'cRnti'})
                print('ABF_BeamMgmtNbm Num rows: ', ABF_BeamMgmtNbm.shape[0])
                #print("Number of rows with NaNs: " + str(np.count_nonzero(ABF_BeamMgmtNbm.isnull())))  

                #ABF_BeamMgmtWbmNbm = pd.read_csv(ran_logs_dir+files[4], sep='\t')
                #ABF_BeamMgmtWbmNbm.drop(ABF_BeamMgmtWbmNbm.columns[-1], axis=1, inplace=True)
                #ABF_BeamMgmtWbmNbm = ABF_BeamMgmtWbmNbm.rename(columns={'CRnti': 'cRnti'})
                #print('ABF_BeamMgmtWbmNbm Num rows: ', ABF_BeamMgmtWbmNbm.shape[0])
                #print("Number of rows with NaNs: " + str(np.count_nonzero(ABF_BeamMgmtWbmNbm.isnull()))) 
                
                print('Done Reading ran logs')
                print('======================================================')
                
                # DEBUG
                # Save the columns to file so we know what all we have 
                #ulSchedGrantEvent.columns.to_series().to_csv('ulSchedGrantEvent_cols.csv', index=False)
                #dlSchedGrantEvent.columns.to_series().to_csv('dlSchedGrantEvent_cols.csv', index=False)
                #ABF_BeamMgmtWbm.columns.to_series().to_csv('ABF_BeamMgmtWbm_cols.csv', index=False)
                #ABF_BeamMgmtNbm.columns.to_series().to_csv('ABF_BeamMgmtNbm_cols.csv', index=False)
        
                # convert epoch timestamp to datatime format 
                ulSchedGrantEvent['timeStamp'] = pd.to_datetime(ulSchedGrantEvent['timeStamp'])
                dlSchedGrantEvent['timeStamp'] = pd.to_datetime(dlSchedGrantEvent['timeStamp'])
                ABF_BeamMgmtWbm['timeStamp'] = pd.to_datetime(ABF_BeamMgmtWbm['timeStamp'])
                ABF_BeamMgmtNbm['timeStamp'] = pd.to_datetime(ABF_BeamMgmtNbm['timeStamp'])
                #ABF_BeamMgmtWbmNbm['timeStamp'] = pd.to_datetime(ABF_BeamMgmtWbmNbm['timeStamp'])
                ul_delay = delay[['t2', 'owd_ul']] # used to be t1 which I think is the transmit time stamp 
                dl_delay = delay[['t4', 'owd_dl']] # used to be t3 which I think is the transmit time stamp 
                ul_delay['t2'] = pd.to_datetime(ul_delay['t2'], unit='ms', origin='unix')
                ul_delay.rename(columns = {'t2':'timeStamp'}, inplace = True)
                dl_delay['t4'] = pd.to_datetime(dl_delay['t4'], unit='ms', origin='unix')
                dl_delay.rename(columns = {'t4':'timeStamp'}, inplace = True)
    
                # set the datetime format timestamp as the index so this is now a series indexed by time
                ulSchedGrantEvent.set_index('timeStamp', inplace=True)
                dlSchedGrantEvent.set_index('timeStamp', inplace=True)
                ABF_BeamMgmtWbm.set_index('timeStamp', inplace=True)
                ABF_BeamMgmtNbm.set_index('timeStamp', inplace=True)
                #ABF_BeamMgmtWbmNbm.set_index('timeStamp', inplace=True)
                ul_delay.set_index('timeStamp', inplace=True)  
                dl_delay.set_index('timeStamp', inplace=True)  
                
                print('Done setting time as index')
                print('======================================================')
                
                # DEBUG
                #print(ulSchedGrantEvent['ueTraceIdMsw'].value_counts())
                #print(dlSchedGrantEvent['ueTraceIdMsw'].value_counts())
                #print(ABF_BeamMgmtWbm['ueTraceIdMsw'].value_counts())
                #print(ABF_BeamMgmtNbm['ueTraceIdMsw'].value_counts())
                
                # Remove entries for user ID 0 and drop some time indicating features 
                ulSchedGrantEvent = ulSchedGrantEvent.loc[ulSchedGrantEvent['ueTraceIdMsw'] != '0x00000000',:].drop(drop_features, axis=1, errors='ignore')
                dlSchedGrantEvent = dlSchedGrantEvent.loc[dlSchedGrantEvent['ueTraceIdMsw'] != '0x00000000',:].drop(drop_features, axis=1, errors='ignore')
                ABF_BeamMgmtWbm = ABF_BeamMgmtWbm.loc[ABF_BeamMgmtWbm['ueTraceIdMsw'] != '0x00000000',: ].drop(drop_features, axis=1, errors='ignore')
                ABF_BeamMgmtNbm = ABF_BeamMgmtNbm.loc[ABF_BeamMgmtNbm['ueTraceIdMsw'] != '0x00000000',:].drop(drop_features, axis=1, errors='ignore')
                #ABF_BeamMgmtWbmNbm = ABF_BeamMgmtWbmNbm.loc[ABF_BeamMgmtWbmNbm['cRnti'] != 0,:].drop(drop_features, axis=1, errors='ignore')
                
                print('Done dropping features from drop_features')
                print('======================================================')
                
                # Remove delay values that are 0. Do not know why these happen, but very infrequently they do 
                print('Number of samples where ul_delay was 0: ', ul_delay[ul_delay['owd_ul'] == 0].shape[0])
                print('Number of samples where dl_delay was 0: ', dl_delay[dl_delay['owd_dl'] == 0].shape[0])
                ul_delay = ul_delay[ul_delay['owd_ul'] > 0]
                dl_delay = dl_delay[dl_delay['owd_dl'] > 0]
                
                print('Done removing 0 valued delay samples')
                print('======================================================')
                
                # ueTraceIdMsw is a consistent identifier for UEs even as they switch to LTE and back (unlike RNTI).   
                if load != '0':
                    load_ue_id=ulSchedGrantEvent.groupby(by=['ueTraceIdMsw'])['tbSizeInBits'].sum().sort_values(ascending=False).index[0]
                    meas_ue_id=ulSchedGrantEvent.groupby(by=['ueTraceIdMsw'])['tbSizeInBits'].sum().sort_values(ascending=False).index[1]
                else:
                    load_ue_id=ulSchedGrantEvent.groupby(by=['ueTraceIdMsw'])['tbSizeInBits'].sum().sort_values(ascending=False).index[1]
                    meas_ue_id=ulSchedGrantEvent.groupby(by=['ueTraceIdMsw'])['tbSizeInBits'].sum().sort_values(ascending=False).index[0]
                    
                print(ulSchedGrantEvent.groupby(by=['ueTraceIdMsw'])['tbSizeInBits'].sum().sort_values(ascending=False))
                print('load_ue_id: ', load_ue_id)
                print('meas_ue_id: ', meas_ue_id)
                print('Done setting meas and load UE IDs')
                print('======================================================')
                
                # Filter out samples for each UE and then drop cRnti as a feature 
                # measurement UE features
                m_ulSchedGrantEvent = ulSchedGrantEvent.loc[ulSchedGrantEvent['ueTraceIdMsw'] == meas_ue_id,:].drop('ueTraceIdMsw', axis=1, errors='ignore')
                m_dlSchedGrantEvent = dlSchedGrantEvent.loc[dlSchedGrantEvent['ueTraceIdMsw'] == meas_ue_id,:].drop('ueTraceIdMsw', axis=1, errors='ignore')
                m_ABF_BeamMgmtWbm = ABF_BeamMgmtWbm.loc[ABF_BeamMgmtWbm['ueTraceIdMsw'] == meas_ue_id,:].drop('ueTraceIdMsw', axis=1, errors='ignore')
                m_ABF_BeamMgmtNbm = ABF_BeamMgmtNbm.loc[ABF_BeamMgmtNbm['ueTraceIdMsw'] == meas_ue_id,:].drop('ueTraceIdMsw', axis=1, errors='ignore')
                #m_ABF_BeamMgmtWbmNbm = ABF_BeamMgmtWbmNbm.loc[ABF_BeamMgmtWbmNbm['ueTraceIdMsw'] == meas_ue_id,:].drop('ueTraceIdMsw', axis=1, errors='ignore')
                # loader UE features 
                ld_ulSchedGrantEvent = ulSchedGrantEvent.loc[ulSchedGrantEvent['ueTraceIdMsw'] == load_ue_id,:].drop('ueTraceIdMsw', axis=1, errors='ignore')
                ld_dlSchedGrantEvent = dlSchedGrantEvent.loc[dlSchedGrantEvent['ueTraceIdMsw'] == load_ue_id,:].drop('ueTraceIdMsw', axis=1, errors='ignore')
                ld_ABF_BeamMgmtWbm = ABF_BeamMgmtWbm.loc[ABF_BeamMgmtWbm['ueTraceIdMsw'] == load_ue_id,:].drop('ueTraceIdMsw', axis=1, errors='ignore')
                ld_ABF_BeamMgmtNbm = ABF_BeamMgmtNbm.loc[ABF_BeamMgmtNbm['ueTraceIdMsw'] == load_ue_id,:].drop('ueTraceIdMsw', axis=1, errors='ignore')
                #ld_ABF_BeamMgmtWbmNbm = ABF_BeamMgmtWbmNbm.loc[ABF_BeamMgmtWbmNbm['ueTraceIdMsw'] == load_ue_id,:].drop('ueTraceIdMsw', axis=1, errors='ignore')
                
                # Create a series with the shifts between NR and LTE by looking when the RNTI has changed 
                m_rnti_change = m_ulSchedGrantEvent['cRnti'] - m_ulSchedGrantEvent['cRnti'].shift(1)
                m_rnti_change_index = m_ulSchedGrantEvent.index
                m_rnti_change[m_rnti_change != 0] = 1
                
                # Then drop the cRnti 
                m_ulSchedGrantEvent = m_ulSchedGrantEvent.drop('cRnti', axis=1, errors='ignore')
                m_dlSchedGrantEvent = m_dlSchedGrantEvent.drop('cRnti', axis=1, errors='ignore')
                m_ABF_BeamMgmtWbm = m_ABF_BeamMgmtWbm.drop('cRnti', axis=1, errors='ignore')
                m_ABF_BeamMgmtNbm = m_ABF_BeamMgmtNbm.drop('cRnti', axis=1, errors='ignore')
                #m_ABF_BeamMgmtWbmNbm = m_ABF_BeamMgmtWbmNbm.drop('cRnti', axis=1, errors='ignore')
                ld_ulSchedGrantEvent = ld_ulSchedGrantEvent.drop('cRnti', axis=1, errors='ignore')
                ld_dlSchedGrantEvent = ld_dlSchedGrantEvent.drop('cRnti', axis=1, errors='ignore')
                ld_ABF_BeamMgmtWbm = ld_ABF_BeamMgmtWbm.drop('cRnti', axis=1, errors='ignore')
                ld_ABF_BeamMgmtNbm = ld_ABF_BeamMgmtNbm.drop('cRnti', axis=1, errors='ignore')
                #ld_ABF_BeamMgmtWbmNbm = ld_ABF_BeamMgmtWbmNbm.drop('cRnti', axis=1, errors='ignore')
                
                print('Done separating samples of meas and load UEs')
                print('======================================================')
                
                # DEBUG ############
                # Tried to understand what are the different values of csiRequest and if I can identify them as 
                # macSdu logs entries, BSR log entries or something like that. 
                # I could not tell what the code 0, 2, 5 are for the csiRequest column  
                #print("m_ulSchedGrantEvent['csiRequest'].value_counts()")
                #print(m_ulSchedGrantEvent['csiRequest'].value_counts())
                #m_ulSchedGrantEvent.loc[ m_ulSchedGrantEvent['csiRequest'] == 0].to_csv('temp_0.csv', index=True)
                #m_ulSchedGrantEvent.loc[ m_ulSchedGrantEvent['csiRequest'] == 2].to_csv('temp_2.csv', index=True)
                #m_ulSchedGrantEvent.loc[ m_ulSchedGrantEvent['csiRequest'] == 5].to_csv('temp_5.csv', index=True)
                #print("ld_ulSchedGrantEvent['csiRequest'].value_counts()")
                #print(ld_ulSchedGrantEvent['csiRequest'].value_counts())
                
                # DEBUG ###############
                # Understanding how many samples are in the ulsched log file as the load changes
                # The number of samples are not uniform and hence taking mean of values especially 
                # for the ones that have a lot of zeros will reduce the accuracy of the value    
                #diff = m_ulSchedGrantEvent.index.to_series().diff()
                #diff = diff.drop(index=diff.index[0]).astype(np.int64) / int(1e6)
                #print('m_ulSchedGrantEvent time_diff')
                #print(diff.describe())
                
                #diff = ld_ulSchedGrantEvent.index.to_series().diff()
                #diff = diff.drop(index=diff.index[0]).astype(np.int64) / int(1e6)
                #print('ld_ulSchedGrantEvent time_diff')
                #print(diff.describe())
                
                #diff = m_dlSchedGrantEvent.index.to_series().diff()
                #diff = diff.drop(index=diff.index[0]).astype(np.int64) / int(1e6)
                #print('m_dlSchedGrantEvent time_diff')
                #print(diff.describe())
                
                #diff = ld_dlSchedGrantEvent.index.to_series().diff()
                #diff = diff.drop(index=diff.index[0]).astype(np.int64) / int(1e6)
                #print('ld_dlSchedGrantEvent time_diff')
                #print(diff.describe())
                
                #diff = m_ABF_BeamMgmtWbm.index.to_series().diff()
                #diff = diff.drop(index=diff.index[0]).astype(np.int64) / int(1e6)
                #print('m_ABF_BeamMgmtWbm time_diff')
                #print(diff.describe())
                
                #diff = ld_ABF_BeamMgmtWbm.index.to_series().diff()
                #diff = diff.drop(index=diff.index[0]).astype(np.int64) / int(1e6)
                #print('ld_ABF_BeamMgmtWbm time_diff')
                #print(diff.describe())
                
                #diff = m_ABF_BeamMgmtNbm.index.to_series().diff()
                #diff = diff.drop(index=diff.index[0]).astype(np.int64) / int(1e6)
                #print('m_ABF_BeamMgmtNbm time_diff')
                #print(diff.describe())
                
                #diff = ld_ABF_BeamMgmtNbm.index.to_series().diff()
                #diff = diff.drop(index=diff.index[0]).astype(np.int64) / int(1e6)
                #print('ld_ABF_BeamMgmtNbm time_diff')
                #print(diff.describe())
 
                # checking if taking mean or count makes a big difference 
                #print('resampled ld_ulSchedGrantEvent macSduInBytes as mean ')
                #print(ld_ulSchedGrantEvent['macSduInBytes'].resample(time_step_size).mean().describe())
                #print('resampled ld_ulSchedGrantEvent macSduInBytes as count ')
                #print((ld_ulSchedGrantEvent['macSduInBytes'].resample(time_step_size).sum()*8/50000).describe())
                
                # DEBUG ############### 
                # How many rows in ul and dl sched info files exist even when mcs/tbs/prbs is 0 for the entry 
                #print('Num of rows in ulSchedGrantEvent: ', ulSchedGrantEvent.shape[0])
                #print('Num of rows with mcsIndex=0: ', ulSchedGrantEvent.loc[ulSchedGrantEvent['mcsIndex'] == 0, 'csiRequest'])
                #print('Num of rows with mcsIndex>0: ', ulSchedGrantEvent.loc[ulSchedGrantEvent['mcsIndex'] > 0, 'csiRequest'])
                #print('Num of rows with numOfPrbs=0: ', ulSchedGrantEvent.loc[ulSchedGrantEvent['numOfPrbs'] == 0,:].shape[0])
                #print('Num of rows with tbSizeInBits=0: ', ulSchedGrantEvent.loc[ulSchedGrantEvent['tbSizeInBits'] == 0,:].shape[0])
                #print('Num of rows with macSduInBytes>0: ', ulSchedGrantEvent.loc[ulSchedGrantEvent['macSduInBytes'] > 0,:].shape[0])
                #print('Num of rows in meas UE with macSduInBytes>0: ', m_ulSchedGrantEvent.loc[m_ulSchedGrantEvent['macSduInBytes'] > 0,:].shape[0])
                #print('Num of rows in loader UE with macSduInBytes>0: ', ld_ulSchedGrantEvent.loc[ld_ulSchedGrantEvent['macSduInBytes'] > 0,:].shape[0])
                #print('Num of rows with postEqSinr0=0: ', ulSchedGrantEvent.loc[ulSchedGrantEvent['postEqSinr0'] == 0,:].shape[0])
                #print('Num of rows with postEqSinr1=0: ', ulSchedGrantEvent.loc[ulSchedGrantEvent['postEqSinr1'] == 0,:].shape[0])
                #print('Num of rows with WBeamRsrpCurrent=0: ', ulSchedGrantEvent.loc[ulSchedGrantEvent['WBeamRsrpCurrent'] == 0,:].shape[0])
                #print('Num of rows with NBeamRsrpCurrent=0: ', ulSchedGrantEvent.loc[ulSchedGrantEvent['NBeamRsrpCurrent'] == 0,:].shape[0])
                #print('Num of rows in dlSchedGrantEvent: ', dlSchedGrantEvent.shape[0])
                #print('Num of rows with DL sinr=0: ', dlSchedGrantEvent.loc[dlSchedGrantEvent['sinr'] == 0,:].shape[0])
                #print('Num of rows with DL cqi=0: ', dlSchedGrantEvent.loc[dlSchedGrantEvent['cqi'] == 0,:].shape[0])
                #print(ulSchedGrantEvent.loc[ulSchedGrantEvent['mcsIndex'] == 0,['mcsIndex','numOfPrbs','tbSizeInBits', 'ACK'] ]) 
                #print(ulSchedGrantEvent.loc[ulSchedGrantEvent['ACK'] == 0,['mcsIndex','numOfPrbs','tbSizeInBits', 'ACK'] ])
                #print('Num of rows in m_dlSchedGrantEvent: ', m_dlSchedGrantEvent.shape[0])
                #print('mcsIndex=0: ', m_dlSchedGrantEvent.loc[m_dlSchedGrantEvent['mcsIndex'] == 0,:].shape[0])
                #print('numOfPrbs=0: ', m_dlSchedGrantEvent.loc[m_dlSchedGrantEvent['numOfPrbs'] == 0,:].shape[0])
                #print('tbSizeInBits=0: ', m_dlSchedGrantEvent.loc[m_dlSchedGrantEvent['tbSizeInBits'] == 0,:].shape[0])                
                
                # Aggregate into arrays the metrics that we would like to aggregate over multiple files 
                aggr_uldelay.extend(ul_delay['owd_ul'])
                aggr_dldelay.extend(dl_delay['owd_dl'])
                aggr_ul_loadUe_measNumOfPrb.extend(ld_ulSchedGrantEvent['measNumOfPrb'])
                aggr_ul_loadUe_numOfPrbs.extend(ld_ulSchedGrantEvent['numOfPrbs'])

                # to aggregate metrics over positions (stationary vs moving) for each load for a view of impact of load on these metrics  
                per_load_ul_delay[ld_id] = per_load_ul_delay[ld_id] + np.mean(ul_delay['owd_ul'])
                per_load_ul_txAtt[ld_id] = per_load_ul_txAtt[ld_id] + np.mean(m_ulSchedGrantEvent['transmissionAttempt'])
                per_load_ul_ack[ld_id] = per_load_ul_ack[ld_id] + np.mean(m_ulSchedGrantEvent['ACK'])
                per_load_ul_buffer_meas[ld_id] = per_load_ul_buffer_meas[ld_id] + np.mean(m_ulSchedGrantEvent['BSRestimate'])
                per_load_ul_buffer_loader[ld_id] = per_load_ul_buffer_loader[ld_id] + np.mean(ld_ulSchedGrantEvent['BSRestimate'])
                
                # Print some stuff I want to see printed
                print('ul delay percentiles: ', np.nanquantile(ul_delay['owd_ul'], 0.10) ,np.nanquantile(ul_delay['owd_ul'], 0.50), 
                      np.nanquantile(ul_delay['owd_ul'], 0.90), np.nanquantile(ul_delay['owd_ul'], 0.97))
                # and save it to a file too like a table
                delay_percentiles_writer.writerow([measUeDev, delayPktSize, position, load, np.nanquantile(ul_delay['owd_ul'], 0.10), np.nanquantile(ul_delay['owd_ul'], 0.50), 
                      np.nanquantile(ul_delay['owd_ul'], 0.90), np.nanquantile(ul_delay['owd_ul'], 0.97)])
                
                # per dev type size aggregation of delay 
                if measUeDev == 'wnc':
                    aggr_uldelay_wnc = np.append(aggr_uldelay_wnc, ul_delay[ul_delay['owd_ul'] < ul_delay_outlier_threshold]['owd_ul'])
                    
                    aggr_ul_m_WBeamRsrp_wnc = np.append(aggr_ul_m_WBeamRsrp_wnc, m_ulSchedGrantEvent['WBeamRsrpCurrent'])
                    aggr_ul_m_NBeamRsrp_wnc = np.append(aggr_ul_m_NBeamRsrp_wnc, m_ulSchedGrantEvent['NBeamRsrpCurrent'])
                    aggr_ul_m_sinrAchievable_wnc = np.append(aggr_ul_m_sinrAchievable_wnc, m_ulSchedGrantEvent['sinrAchievable'])
                    aggr_ul_m_BSRestimate_wnc = np.append(aggr_ul_m_BSRestimate_wnc, m_ulSchedGrantEvent['BSRestimate'])
                    aggr_ul_ld_macSduInBytes_wnc = np.append(aggr_ul_ld_macSduInBytes_wnc, ld_ulSchedGrantEvent['macSduInBytes'])
                    aggr_dl_m_sinr_wnc = np.append(aggr_dl_m_sinr_wnc, m_dlSchedGrantEvent['sinr'])
                    
                elif measUeDev == 'sony':
                    aggr_uldelay_sony = np.append(aggr_uldelay_sony, ul_delay[ul_delay['owd_ul'] < ul_delay_outlier_threshold]['owd_ul'])
                    
                    aggr_ul_m_WBeamRsrp_sony = np.append(aggr_ul_m_WBeamRsrp_sony, m_ulSchedGrantEvent['WBeamRsrpCurrent'])
                    aggr_ul_m_NBeamRsrp_sony = np.append(aggr_ul_m_NBeamRsrp_sony, m_ulSchedGrantEvent['NBeamRsrpCurrent'])
                    aggr_ul_m_sinrAchievable_sony = np.append(aggr_ul_m_sinrAchievable_sony, m_ulSchedGrantEvent['sinrAchievable'])
                    aggr_ul_m_BSRestimate_sony = np.append(aggr_ul_m_BSRestimate_sony, m_ulSchedGrantEvent['BSRestimate'])
                    aggr_ul_ld_macSduInBytes_sony = np.append(aggr_ul_ld_macSduInBytes_sony, ld_ulSchedGrantEvent['macSduInBytes'])
                    aggr_dl_m_sinr_sony = np.append(aggr_dl_m_sinr_sony, m_dlSchedGrantEvent['sinr'])
                
                # per packet size aggregation of delay 
                if delayPktSize == '1400':
                    aggr_uldelay_1400.extend(ul_delay[ul_delay['owd_ul'] < ul_delay_outlier_threshold]['owd_ul'])
                elif delayPktSize == '100':
                    aggr_uldelay_100.extend(ul_delay[ul_delay['owd_ul'] < ul_delay_outlier_threshold]['owd_ul'])
                
                # per stationary/moving combined over all stationary positions aggregation of delay
                if position == '0' :
                    aggr_uldelay_moving.extend(ul_delay[ul_delay['owd_ul'] < ul_delay_outlier_threshold]['owd_ul'])
                else:
                    aggr_uldelay_static.extend(ul_delay[ul_delay['owd_ul'] < ul_delay_outlier_threshold]['owd_ul'])
                
                # over all samples 
                aggr_uldelay.extend(ul_delay[ul_delay['owd_ul'] < ul_delay_outlier_threshold]['owd_ul'])
                
                # Aggregating for the prescheduling demoonstation plot 
                if position != '0' and load == '10':
                    aggr_mcs_stat_load10 = np.append(aggr_mcs_stat_load10, ld_ulSchedGrantEvent['mcsIndex'])
                    aggr_sinr0_stat_load10 = np.append(aggr_sinr0_stat_load10, ld_ulSchedGrantEvent['postEqSinr0'])
                    aggr_sinr1_stat_load10 = np.append(aggr_sinr1_stat_load10, ld_ulSchedGrantEvent['postEqSinr1'])
                    aggr_bytes_stat_load10 = np.append(aggr_bytes_stat_load10, ld_ulSchedGrantEvent['tbSizeInBits']/8)
                    aggr_prbs_stat_load10 = np.append(aggr_prbs_stat_load10, ld_ulSchedGrantEvent['numOfPrbs'])
                if position != '0' and load == '60':
                    if delayPktSize == '1400':
                        aggr_mcs_stat_load60_1400B_m = np.append(aggr_mcs_stat_load60_1400B_m, m_ulSchedGrantEvent['mcsIndex'])
                    else: #delayPktSize == '100'
                        aggr_mcs_stat_load60_100B_m = np.append(aggr_mcs_stat_load60_100B_m, m_ulSchedGrantEvent['mcsIndex'])  
                    aggr_mcs_stat_load60 = np.append(aggr_mcs_stat_load60, ld_ulSchedGrantEvent['mcsIndex'])
                    aggr_sinr0_stat_load60 = np.append(aggr_sinr0_stat_load60, ld_ulSchedGrantEvent['postEqSinr0'])
                    aggr_sinr1_stat_load60 = np.append(aggr_sinr1_stat_load60, ld_ulSchedGrantEvent['postEqSinr1'])
                    aggr_bytes_stat_load60 = np.append(aggr_bytes_stat_load60, ld_ulSchedGrantEvent['tbSizeInBits']/8)
                    aggr_prbs_stat_load60 = np.append(aggr_prbs_stat_load60, ld_ulSchedGrantEvent['numOfPrbs'])
                
                # Aggregate standard deviation over resample windows to compare how variant select features are 
                # between stat and moving and between device types
                if (position == '0' and measUeDev == 'wnc' and delayPktSize == '1400' and load == '30'):
                    wind_q10to90_ulDelay_mov_wnc = ul_delay.resample('1000ms').quantile(0.90) - ul_delay.resample('1000ms').quantile(0.10)
                    wind_q50_ulDelay_mov_wnc = ul_delay.resample('1000ms').quantile(0.50)
                    #wind_mean_dlsinr_mov_wnc = m_dlSchedGrantEvent.resample('1000ms').mean()
                    #wind_sd_dlsinr_mov_wnc = m_dlSchedGrantEvent.resample('1000ms').std()    
                elif (position == '1' and measUeDev == 'wnc' and delayPktSize == '1400' and load == '30'):
                    wind_q10to90_ulDelay_stat_wnc = ul_delay.resample('1000ms').quantile(0.90)- ul_delay.resample('1000ms').quantile(0.10)
                    wind_q50_ulDelay_stat_wnc = ul_delay.resample('1000ms').quantile(0.50)
                    #wind_mean_dlsinr_stat_wnc = m_dlSchedGrantEvent.resample('1000ms').mean()
                    #wind_sd_dlsinr_stat_wnc = m_dlSchedGrantEvent.resample('1000ms').std()
                elif (position == '0' and measUeDev == 'sony' and delayPktSize == '1400' and load == '30'):
                    wind_q10to90_ulDelay_mov_sony = ul_delay.resample('1000ms').quantile(0.90) - ul_delay.resample('1000ms').quantile(0.10)
                    wind_q50_ulDelay_mov_sony = ul_delay.resample('1000ms').quantile(0.50)
                    #wind_mean_dlsinr_mov_sony = m_dlSchedGrantEvent.resample('1000ms').mean()
                    #wind_sd_dlsinr_mov_sony = m_dlSchedGrantEvent.resample('1000ms').std()
                elif (position == '1' and measUeDev == 'sony' and delayPktSize == '1400' and load == '30'):
                    wind_q10to90_ulDelay_stat_sony = ul_delay.resample('1000ms').quantile(0.90) - ul_delay.resample('1000ms').quantile(0.10)
                    wind_q50_ulDelay_stat_sony = ul_delay.resample('1000ms').quantile(0.50)
                    #wind_mean_dlsinr_stat_sony = m_dlSchedGrantEvent.resample('1000ms').mean()
                    #wind_sd_dlsinr_stat_sony = m_dlSchedGrantEvent.resample('1000ms').std()
                 
                print('Done aggregating raw metrics over files for plots/analysis') 
                print('======================================================')
                
                #=================================================  Resample  ==================================================
                if wind_parse_dataset:
                    # testing chipset type 
                    print('meas UE: ', m_ulSchedGrantEvent['chipsetType'].value_counts())
                    print('loader UE: ', ld_ulSchedGrantEvent['chipsetType'].value_counts())
                        
                    # I am opnly going to consider the ul sched log file when using n step history because I am using 10 ms windows 
                    # and the other logs do not have samples that frequent
                    if n_step_history:                        
                        # create past steps and aggregate them as features 
                        # Before resampling using mean.. some metrics need to be summed instead, so do that separately first 
                        m_sum_mac_bytes = m_ulSchedGrantEvent['macSduInBytes'].resample(feat_step_size).sum().resample(time_step_size).agg(list)
                        m_sum_tb_bits = m_ulSchedGrantEvent['tbSizeInBits'].resample(feat_step_size).sum().resample(time_step_size).agg(list)
                        ld_sum_mac_bytes = ld_ulSchedGrantEvent['macSduInBytes'].resample(feat_step_size).sum().resample(time_step_size).agg(list)
                        ld_sum_tb_bits = ld_ulSchedGrantEvent['tbSizeInBits'].resample(feat_step_size).sum().resample(time_step_size).agg(list)
                        
                        # Resample into windows of step size and mean over the window
                        reSamp_m_ulDelay = ul_delay.resample(time_step_size).mean()
                        reSamp_m_dlDelay = dl_delay.resample(time_step_size).mean()
                        
                        reSamp_m_ulSched = m_ulSchedGrantEvent.resample(feat_step_size).mean().resample(time_step_size).agg(list)
                        reSamp_ld_ulSched = ld_ulSchedGrantEvent.resample(feat_step_size).mean().resample(time_step_size).agg(list)
                        
                        # Find the time range over which to crop the logs
                        sampled_log_start_time = max(reSamp_m_ulDelay.index[0], 
                                                     reSamp_m_dlDelay.index[0],
                                                     reSamp_m_ulSched.index[0],
                                                     reSamp_ld_ulSched.index[0])
                        sampled_log_end_time = min(reSamp_m_ulDelay.index[-1], 
                                                   reSamp_m_dlDelay.index[-1],
                                                   reSamp_m_ulSched.index[-1],
                                                   reSamp_ld_ulSched.index[-1])
                        
                        # crop into this time period to make everything uniform so we can combine them together
                        reSamp_m_ulSched = reSamp_m_ulSched[sampled_log_start_time : sampled_log_end_time].iloc[1:]
                        reSamp_ld_ulSched = reSamp_ld_ulSched[sampled_log_start_time : sampled_log_end_time].iloc[1:]
                        reSamp_m_ulDelay = reSamp_m_ulDelay[sampled_log_start_time : sampled_log_end_time].iloc[1:]
                        reSamp_m_dlDelay = reSamp_m_dlDelay[sampled_log_start_time : sampled_log_end_time].iloc[1:]
                        
                        # open up the columns 
                        print('before adding the step cols: ', reSamp_m_ulSched.shape)
                        
                        for col in reSamp_m_ulSched.columns:
                            # add the list as multiple colums 
                            step_cols = [col+'_'+str(i) for i in range(num_steps)]
                            reSamp_m_ulSched[step_cols] = reSamp_m_ulSched[col].str[0:num_steps][0]
                            reSamp_ld_ulSched[step_cols] = reSamp_ld_ulSched[col].str[0:num_steps][0]
                            #print('After adding one set of step cols: ', reSamp_m_ulSched.shape)
                            # then delete the col with the list
                            reSamp_m_ulSched = reSamp_m_ulSched.drop([col], axis=1)
                            reSamp_ld_ulSched = reSamp_ld_ulSched.drop([col], axis=1)
                            #print('After dropping the OG col: ', reSamp_m_ulSched.shape)
                        
                        # replace using the sum metrics that we computed before
                        reSamp_ld_ulSched['macSduInBytes'] = ld_sum_mac_bytes
                        reSamp_ld_ulSched['tbSizeInBits'] = ld_sum_tb_bits

                        reSamp_m_ulSched['macSduInBytes'] = m_sum_mac_bytes
                        reSamp_m_ulSched['tbSizeInBits'] = m_sum_tb_bits
                        # replace using the sum metrics that we computed before
                        print(reSamp_m_ulSched.columns)
                        print(reSamp_ld_ulSched.columns)
                        
                        # concatenate column wise the measurement UE and loader UE featutres for all log files     
                        # WARNING: if the columns are mismatched then those will just be moved to the end and concatenated
                        this_run_data = pd.concat([reSamp_m_ulSched.add_prefix('ul_'), 
                                                   reSamp_ld_ulSched.add_prefix('loadUe_').add_prefix('ul_'), 
                                                   reSamp_m_ulDelay, reSamp_m_dlDelay], axis=1)
                        print('Number of NAs in this run wind combined log: ', this_run_data.shape[0] - this_run_data.dropna().shape[0])
                                        
                        # Drop the samples where a leg switch has happened for the meas UE only since that is when delay values are over NR  
                        # Add 'ul_loadUe_NBeamRsrpCurrent' to the drop subset if you want to drop even if loader UE disconnects 
                        print('Number of NAs dropped because of meas UE leg switch: ', 
                              this_run_data.shape[0] - this_run_data.dropna(subset=['ul_NBeamRsrpCurrent_1']).shape[0])
                        # Drop the values that correspond to it going over LTE. We identify this by seeing if there are any NR ulSched RAN metrics being logged
                        this_run_data = this_run_data.dropna(subset=['ul_NBeamRsrpCurrent_1'])
                        
                    #=================
                    else:
                        # Before resampling using mean.. some metrics need to be summed instead, so do that separately first 
                        m_sum_mac_bytes = m_ulSchedGrantEvent['macSduInBytes'].resample(time_step_size).sum()
                        m_sum_tb_bits = m_ulSchedGrantEvent['tbSizeInBits'].resample(time_step_size).sum()
                        # Resample into windows of step size and mean over the window
                        reSamp_m_ulSched = m_ulSchedGrantEvent.resample(time_step_size).mean()
                        reSamp_m_dlSched = m_dlSchedGrantEvent.resample(time_step_size).mean()
                        reSamp_m_BeamMgmtWbm = m_ABF_BeamMgmtWbm.resample(time_step_size).mean()
                        reSamp_m_BeamMgmtNbm = m_ABF_BeamMgmtNbm.resample(time_step_size).mean()
                        #reSamp_m_BeamMgmtWbmNbm = m_ABF_BeamMgmtWbmNbm.resample(time_step_size).mean()
                        # replace using the sum metrics that we computed before
                        reSamp_m_ulSched['macSduInBytes'] = m_sum_mac_bytes
                        reSamp_m_ulSched['tbSizeInBits'] = m_sum_tb_bits

                        # Before resampling using mean.. some metrics need to be summed instead, so do that separately first 
                        ld_sum_mac_bytes = ld_ulSchedGrantEvent['macSduInBytes'].resample(time_step_size).sum()
                        ld_sum_tb_bits = ld_ulSchedGrantEvent['tbSizeInBits'].resample(time_step_size).sum()
                        reSamp_ld_ulSched = ld_ulSchedGrantEvent.resample(time_step_size).mean()
                        reSamp_ld_dlSched = ld_dlSchedGrantEvent.resample(time_step_size).mean()
                        reSamp_ld_BeamMgmtWbm = ld_ABF_BeamMgmtWbm.resample(time_step_size).mean()
                        reSamp_ld_BeamMgmtNbm = ld_ABF_BeamMgmtNbm.resample(time_step_size).mean()
                        #reSamp_ld_BeamMgmtWbmNbm = ld_ABF_BeamMgmtWbmNbm.resample(time_step_size).mean()
                        # replace using the sum metrics that we computed before
                        reSamp_ld_ulSched['macSduInBytes'] = ld_sum_mac_bytes
                        reSamp_ld_ulSched['tbSizeInBits'] = ld_sum_tb_bits

                        reSamp_m_ulSched['macSduInBytes'] = m_sum_mac_bytes
                        reSamp_m_ulSched['tbSizeInBits'] = m_sum_tb_bits
                        # replace using the sum metrics that we computed before

                        reSamp_m_ulDelay = ul_delay.resample(time_step_size).mean()
                        reSamp_m_dlDelay = dl_delay.resample(time_step_size).mean()

                        # Find the time range over which to crop the logs
                        sampled_log_start_time = max(reSamp_m_dlDelay.index[0], 
                                                     reSamp_m_ulSched.index[0], 
                                                     reSamp_m_dlSched.index[0])
                        sampled_log_end_time = min(reSamp_m_dlDelay.index[-1], 
                                                   reSamp_m_ulSched.index[-1], 
                                                   reSamp_m_dlSched.index[-1])

                        # Crop all frames into this time range
                        reSamp_m_ulSched = reSamp_m_ulSched[sampled_log_start_time : sampled_log_end_time]
                        reSamp_m_dlSched = reSamp_m_dlSched[sampled_log_start_time : sampled_log_end_time]
                        reSamp_m_BeamMgmtWbm = reSamp_m_BeamMgmtWbm[sampled_log_start_time : sampled_log_end_time]
                        reSamp_m_BeamMgmtNbm = reSamp_m_BeamMgmtNbm[sampled_log_start_time : sampled_log_end_time]
                        #reSamp_m_BeamMgmtWbmNbm = reSamp_m_BeamMgmtWbmNbm[sampled_log_start_time : sampled_log_end_time]

                        reSamp_ld_ulSched = reSamp_ld_ulSched[sampled_log_start_time : sampled_log_end_time]
                        reSamp_ld_dlSched = reSamp_ld_dlSched[sampled_log_start_time : sampled_log_end_time]
                        reSamp_ld_BeamMgmtWbm = reSamp_ld_BeamMgmtWbm[sampled_log_start_time : sampled_log_end_time]
                        reSamp_ld_BeamMgmtNbm = reSamp_ld_BeamMgmtNbm[sampled_log_start_time : sampled_log_end_time]
                        #reSamp_ld_BeamMgmtWbmNbm = reSamp_ld_BeamMgmtWbmNbm[sampled_log_start_time : sampled_log_end_time]           

                        reSamp_m_ulDelay = reSamp_m_ulDelay[sampled_log_start_time : sampled_log_end_time]      
                        reSamp_m_dlDelay = reSamp_m_dlDelay[sampled_log_start_time : sampled_log_end_time]
                    
                        print('Before: num samples in wind ul delay log: ', reSamp_m_ulDelay.shape[0])
                        print('start, stop time: ', reSamp_m_ulDelay.index[0], reSamp_m_ulDelay.index[-1])
                        print('Number of NAs in wind ul delay log: ', reSamp_m_ulDelay.shape[0] - reSamp_m_ulDelay.dropna().shape[0])

                        # Print to see how many NAs are in each type of log for selected window size
                        print('Number of NAs in this run wind reSamp_m_ulSched log: ', reSamp_m_ulSched.shape[0] - reSamp_m_ulSched.dropna().shape[0])
                        #print('Number of NAs in this run wind reSamp_m_dlSched log: ', reSamp_m_dlSched.shape[0] - reSamp_m_dlSched.dropna().shape[0])
                        print('Number of NAs in this run wind reSamp_m_BeamMgmtWbm log: ', reSamp_m_BeamMgmtWbm.shape[0] - reSamp_m_BeamMgmtWbm.dropna().shape[0])
                        print('Number of NAs in this run wind reSamp_m_BeamMgmtNbm log: ', reSamp_m_BeamMgmtNbm.shape[0] - reSamp_m_BeamMgmtNbm.dropna().shape[0])
                        print('Number of NAs in this run wind reSamp_ld_ulSched log: ', reSamp_ld_ulSched.shape[0] - reSamp_ld_ulSched.dropna().shape[0])
                        #print('Number of NAs in this run wind reSamp_ld_dlSched log: ', reSamp_ld_dlSched.shape[0] - reSamp_ld_dlSched.dropna().shape[0])
                        print('Number of NAs in this run wind reSamp_ld_BeamMgmtWbm log: ', reSamp_ld_BeamMgmtWbm.shape[0] - reSamp_ld_BeamMgmtWbm.dropna().shape[0])
                        print('Number of NAs in this run wind reSamp_ld_BeamMgmtNbm log: ', reSamp_ld_BeamMgmtNbm.shape[0] - reSamp_ld_BeamMgmtNbm.dropna().shape[0])
                        print('Number of NAs in this run wind reSamp_m_ulDelay log: ', reSamp_m_ulDelay.shape[0] - reSamp_m_ulDelay.dropna().shape[0])
                        print('Number of NAs in this run wind reSamp_m_dlDelay log: ', reSamp_m_dlDelay.shape[0] - reSamp_m_dlDelay.dropna().shape[0])
                    
                        # concatenate column wise the measurement UE and loader UE featutres for all log files     
                        # WARNING: if the columns are mismatched then those will just be moved to the end and concatenated
                        this_run_data = pd.concat([reSamp_m_ulSched.add_prefix('ul_'), 
                                                   #reSamp_m_dlSched.add_prefix('dl_'), 
                                                   reSamp_m_BeamMgmtWbm.add_prefix('wbm_'), 
                                                   reSamp_m_BeamMgmtNbm.add_prefix('nbm_'), 
                                                   #reSamp_m_BeamMgmtWbmNbm.add_prefix('wbmnbm_'), 
                                                   reSamp_ld_ulSched.add_prefix('loadUe_').add_prefix('ul_'), 
                                                   #reSamp_ld_dlSched.add_prefix('loadUe_').add_prefix('dl_'),
                                                   reSamp_ld_BeamMgmtWbm.add_prefix('loadUe_').add_prefix('wbm_'), 
                                                   reSamp_ld_BeamMgmtNbm.add_prefix('loadUe_').add_prefix('nbm_'), 
                                                   #reSamp_ld_BeamMgmtWbmNbm.add_prefix('loadUe_').add_prefix('wbmnbm_'), 
                                                   reSamp_m_ulDelay, reSamp_m_dlDelay], axis=1)
                        # Aggregate into an array the delays over LTE to plot them later 
                        if measUeDev == 'wnc':
                            wind_aggr_uldelay_lte_wnc = np.append(wind_aggr_uldelay_lte_wnc, 
                                                                  this_run_data[this_run_data['ul_NBeamRsrpCurrent'].isnull()]['owd_ul'])
                        else:
                            wind_aggr_uldelay_lte_sony = np.append(wind_aggr_uldelay_lte_sony, 
                                                                   this_run_data[this_run_data['ul_NBeamRsrpCurrent'].isnull()]['owd_ul'])

                        print('Number of NAs in this run wind combined log: ', this_run_data.shape[0] - this_run_data.dropna().shape[0])

                        # Drop the samples where a leg switch has happened for the meas UE only since that is when delay values are over NR  
                        # Add 'ul_loadUe_NBeamRsrpCurrent' to the drop subset if you want to drop even if loader UE disconnects 
                        print('Number of NAs dropped because of meas UE leg switch: ', 
                              this_run_data.shape[0] - this_run_data.dropna(subset=['ul_NBeamRsrpCurrent']).shape[0])
                        # Drop the values that correspond to it going over LTE. We identify this by seeing if there are any NR ulSched RAN metrics being logged
                        this_run_data = this_run_data.dropna(subset=['ul_NBeamRsrpCurrent'])    
                    
                    # -------- end of else over if step_history
                    
                    # Add columns for position and load info 
                    this_run_data['position'] = [int(position) for i in range(this_run_data.shape[0])]
                    this_run_data['load_Mbps'] = [int(load) for i in range(this_run_data.shape[0])]
                    
                    # Drop some features that are not in all files  
                    # This is done to accomodate when we have multiple steps of the same col
                    to_drop = [col for col in this_run_data.columns if any(sub_col in col for sub_col in drop_features_after_processing)]
                    print('to_drop due to features not being in all files', to_drop)
                    this_run_data = this_run_data.drop(to_drop, axis=1, errors='ignore')
                    print('to_drop due to constant and non-gen features: len: ', len(drop_cols))
                    this_run_data = this_run_data.drop(drop_cols, axis=1, errors='ignore')
                     
                    # I am choosing to keep the rest of the NAs in the dtaset so that the model can decide what to do with it. 
                    # concatenate row wise to the variable that is being aggregated over
                    dataset = pd.concat([dataset, this_run_data], axis=0)
                    
                    print('Done resampling data into windows') 
                    print('======================================================')
                    
                #==================================== end of if wind_parse_dataset ================================================
                
                # If I want to see plots for each file without any aggregation over different files
                # Here we have plots using raw data as well as plots using resampled/windowed data 
                # Although.. plotting over windowed metrics can also be done outside of this loop in a cell below 
                if plot_single_file_metrics:
                    # Raw data plots 
                    
                    # Tried to plot the RSRP with the leg switch events overlayed, but realised that 
                    # all switches are not receorded so this is not a reliable way to identify the LTE samples 
                    
                    #plt.figure(figsize=(20,5))
                    #plt.plot(m_ulSchedGrantEvent.index, m_ulSchedGrantEvent['NBeamRsrpCurrent'], 'r.', label='meas UE')
                    #plt.plot(ld_ulSchedGrantEvent.index, ld_ulSchedGrantEvent['NBeamRsrpCurrent'], 'g.', label='load UE')
                    #try:
                    #    legSwitch = pd.read_csv(ran_logs_dir+files[4], sep='\t')
                    #    legSwitch.drop(legSwitch.columns[-1], axis=1, inplace=True)
                    #    legSwitch = legSwitch.rename(columns={'CRnti': 'cRnti'})
                    #    print('legSwitch Num rows: ', legSwitch.shape[0])
                    #    print("Number of rows with NaNs: " + str(np.count_nonzero(legSwitch.isnull()))) 
                    #    legSwitch['timeStamp'] = pd.to_datetime(legSwitch['timeStamp'])
                    #    legSwitch.set_index('timeStamp', inplace=True)
                    #    #legSwitch = legSwitch[(legSwitch['ueTraceIdMsw'] == load_ue_id) | (legSwitch['ueTraceIdMsw'] == meas_ue_id)]
                    #    plt.stem([legSwitch.index[0], legSwitch.index[-1]], np.ones(2)*100, bottom=0)
                    #except FileNotFoundError:
                    #    print('File Not Found')
                    #plt.title('Pos: '+str(position)+' Chip: '+str(measUeDev)+' Pkt:'+delayPktSize+' Load:'+load)
                    #plt.xlabel('UE NBeamRsrpCurrent')
                    #plt.legend()
                    #plt.show()

                    # Histogram of raw values for this file 
                    
                    # Substitute with m_ulSchedGrantEvent['mcsIndex'] m_dlSchedGrantEvent['cqi'] 
                    # m_ulSchedGrantEvent['NBeamRsrpCurrent'] etc. and set range if needed to get this file histograms plots 
                    #sns.set(rc={'figure.figsize':(16,4)}, font_scale = 1.5)
                    #plt.figure()
                    #plt.hist(ul_delay['owd_ul'], bins=50, edgecolor='k')
                    #plt.title('Pos: '+str(position)+' Chip: '+str(measUeDev)+' Pkt:'+delayPktSize)
                    #plt.xlabel('UL Delay (ms)')
                    #plt.yticks([])
                    #plt.show()
                    
                    # Timeseries overlay of UL delay and select metrics 
                    
                    # measuremnet UE BSR
                    if (position == '0'):
                        print('Raw delay and BSRestimate samples, LTE NOT removed')
                        sns.set(rc={'figure.figsize':(20,4)}, font_scale = 1.5)
                        fig, ax1 = plt.subplots(figsize=(20, 4))
                        ax2 = ax1.twinx()
                        ax1.plot(ul_delay.index, ul_delay['owd_ul'], 'b*-', label='uplink delay')
                        ax1.set_ylabel('Uplink delay (ms)')
                        ax1.legend(loc='center left')
                        ax2.plot(m_ulSchedGrantEvent.index, m_ulSchedGrantEvent['BSRestimate'], 'r.', label='buffer occupancy')
                        #ax2.stem(m_rnti_change_index, m_rnti_change*max(m_ulSchedGrantEvent['BSRestimate']), markerfmt='k.', linefmt='k-', basefmt=None)
                        ax2.set_xlabel('Time')
                        ax2.set_ylabel('Meas. UE buff (bits)')
                        ax2.legend(loc='center right')
                        # I am likely to save only one of all the plots generated since this is per file, 
                        # so select and only save the one I want
                        if (measUeDev == 'wnc' and delayPktSize == '1400' and load == '30'):
                            plt.savefig('plots_for_paper/timeseries raw uldelay BSR overlay wnc.pdf')
                            plt.savefig('plots_for_paper/timeseries raw uldelay BSR overlay wnc.png')
                        elif (measUeDev == 'sony' and delayPktSize == '1400' and load == '30'):
                            plt.savefig('plots_for_paper/timeseries raw uldelay BSR overlay sony.pdf')
                            plt.savefig('plots_for_paper/timeseries raw uldelay BSR overlay sony.png')
                        plt.show()
                        
                        print('Windowed delay and BSRestimate samples, LTE removed')
                        sns.set(rc={'figure.figsize':(20,4)}, font_scale = 1.5)
                        fig, ax1 = plt.subplots(figsize=(20, 4))
                        ax2 = ax1.twinx()
                        ax1.plot(this_run_data.index, this_run_data['owd_ul'], 'b*-', label='uplink delay')
                        ax1.set_ylabel('Uplink delay (ms)')
                        ax1.legend(loc='center left')
                        ax2.plot(this_run_data.index, this_run_data['ul_BSRestimate'], 'r.', label='buffer occupancy')
                        #ax2.stem(m_rnti_change_index, m_rnti_change*max(m_ulSchedGrantEvent['BSRestimate']), markerfmt='k.', linefmt='k-', basefmt=None)
                        ax2.set_xlabel('Time')
                        ax2.set_ylabel('Meas. UE buff (bits)')
                        ax2.legend(loc='center right')
                        if (measUeDev == 'wnc' and delayPktSize == '1400' and load == '30'):
                            plt.savefig('plots_for_paper/timeseries wind uldelay BSR overlay wnc.pdf')
                            plt.savefig('plots_for_paper/timeseries wind uldelay BSR overlay wnc.png')
                        elif (measUeDev == 'sony' and delayPktSize == '1400' and load == '30'):
                            plt.savefig('plots_for_paper/timeseries wind uldelay BSR overlay sony.pdf')
                            plt.savefig('plots_for_paper/timeseries wind uldelay BSR overlay sony.png')
                        plt.show()

                    #========== Resampled/windowed data plots =======
                    
                    # Timeseries overlay of UL delay and meas UE ACK rate
                    # ACK rate is a bulk metric and needs to be seen in windows, hence we use the resampled data 
                    if (position == '0'):
                        print('Windowed delay and ACK rate samples, LTE removed')
                        sns.set(rc={'figure.figsize':(20,4)}, font_scale = 1.5)
                        fig, ax1 = plt.subplots(figsize=(20, 4))
                        ax2 = ax1.twinx()
                        ax1.plot(this_run_data.index, this_run_data['owd_ul'], 'b*-', label='uplink delay')
                        ax1.set_ylabel('Uplink delay (ms)')
                        ax1.legend(loc='center left')
                        ax2.plot(this_run_data.index, this_run_data['ul_ACK'], 'r.', label='ACK rate')
                        ax2.set_xlabel('Time')
                        ax2.set_ylabel('Meas. UE ACK rate')
                        ax2.legend(loc='center right')
                        # I am likely to save only one of all the plots generated since this is per file, 
                        # so select and only save the one I want
                        if (measUeDev == 'wnc' and delayPktSize == '1400' and load == '30'):
                            plt.savefig('plots_for_paper/timeseries wind uldelay ACKrate overlay wnc.pdf')
                            plt.savefig('plots_for_paper/timeseries wind uldelay ACKrate overlay wnc.png')
                        elif (measUeDev == 'sony' and delayPktSize == '1400' and load == '30'):
                            plt.savefig('plots_for_paper/timeseries wind uldelay ACKrate overlay sony.pdf')
                            plt.savefig('plots_for_paper/timeseries wind uldelay ACKrate overlay sony.png')
                        plt.show()

                    print('Done plotting metrics for this file')
                    print('======================================================')
                print('Completed this iteration in ', time.time() - iteration_start_time)
                #====================================================================================
            # end load 
        # end position
        
        if create_dataset:
            # Save the dataset
            dataset.to_csv('parsed_data/dataset_'+measUeDev+'_delayPktSize_'+delayPktSize+
                           '_timeStepSize_'+time_step_size+'.csv', index=True)
            print('Done writing the dataset into a csv file')
            print('======================================================')
        
    # end measUeDev 
# end delayPktSize
f.close()
print('')
print('')
elapsed_time = time.time() - start_time
print('Completed in ', elapsed_time)
print('===============================  DONE  ===================================')
